### Run the following chunk of code to import any necessary libraries or packages required to run the rest of the code.

In [ ]:
import pandas as pd
import numpy as np
import hoomd, hoomd.md as md
from hoomd import azplugins
import gsd, gsd.hoomd, gsd.pygsd
import warnings
import IPython
import packaging.version
import math
import sys

### Run the following chunk of code to initialize the simulation parameters for the simulation.

The following code initializes the parameters of the simulation that will be used throughout. The first category of parameters are those that we explicitly vary in our experiments. The second set of parameters are those that we did not actively vary, but could in principle be varied in experiments. The final category of parameters are those calculated based upon those defined in the first two categories. 

Note, the user will need to adjust their directory structure to match that of the code, or change the code directory variables to match the users directory structure. 

In [ ]:
base_Directory = ""
working_Input_Directory = base_Directory + "Inputs/"

# These are properties we actively varied in experiments.
phos = "0_x"

working_Output_Directory = base_Directory + "Outputs/" + phos.replace("_","") + "_Phos/"

protein_Concentration = 19 #uM
x_Clones = 5
y_Clones = 5
z_Clones = 10
salt_Concentration = 0.000 #M

# These are properties the user may wish to change for experimental purposes, but were kept constant for our paper results.
time_Step = 0.01 #ps
resizing_Steps = 10000
production_Steps = 5*10**8
temperature = 295 #Kelvin
harmonic_Bond_Length = 0.38 #nm
yukawa_Cutoff_Distance = 3.5 #nm
ashbaugh_Cutoff_Distance = 2.0 #nm
LJ_Epsilon_Base = 0.1875 #kCal/mol
LJ_Epsilon_Max = 0.1905 #kCal/mol
monopotassium_Phosphate_Concentration = 0.006085 #mol/L
dipotassium_Phosphate_Concentration = 0.01392 #mol/L
box_Size = 60 #nm
slab_Z_Length = 5800 #nm
harmonic_Spring_Constant = 8305
squish_Box_Z = 5*box_Size

# These are properties which are either physical constants, or values which are determined via calculations using previously 
# defined variables.
objects_Per_Mole = 6.02*10**(23)
boltzmann_Constant = 1.38*10**(-23) #J/K
boltzmann_Constant_Converted = boltzmann_Constant*(1/(1*10**(3)))*objects_Per_Mole # kJ/mol K
water_Dielectric_Constant = 78.4
electron_Charge = 1.602*10**(-19)
vacuum_Permittivity = 8.85*10**(-12) #C^2/(Nm^2)
vacuum_Permittivity_Converted = vacuum_Permittivity*((1/(1*10**9))*((1*10**3)/1)*(1/(objects_Per_Mole))) #C^2 mol/kJ nm
yukawa_Epsilon_Scaling = (electron_Charge)**(2)/(4*np.pi*water_Dielectric_Constant*vacuum_Permittivity_Converted) #kJ nm/mol
LJ_Potential_Epsilon = LJ_Epsilon_Base + ((LJ_Epsilon_Max - LJ_Epsilon_Base)/1.250)*salt_Concentration#kCal/mol
LJ_Potential_Epsilon_Converted = LJ_Potential_Epsilon*(4.184/1) #kJ/mol
bjerrum_Length = (electron_Charge**2)/(4*np.pi*vacuum_Permittivity*water_Dielectric_Constant*boltzmann_Constant*temperature) #m
bjerrum_Length_Converted = bjerrum_Length*((1*10**(9))/(1)) #nm
debye_Sum_Scaling = bjerrum_Length_Converted*4*np.pi
buffer_Sum = dipotassium_Phosphate_Concentration*(1**2) + dipotassium_Phosphate_Concentration*(1**2) + \
dipotassium_Phosphate_Concentration*((-1)**2) + dipotassium_Phosphate_Concentration*((-1)**2) + \
monopotassium_Phosphate_Concentration*(1**2) + monopotassium_Phosphate_Concentration*((-1)**2) #mol/L
buffer_Sum_Converted = buffer_Sum*objects_Per_Mole*((1)/(1*10**(24))) #ions/nm^3
salt_Sum = salt_Concentration*(1**2) + salt_Concentration*((-1)**2)
salt_Sum_Converted = salt_Sum*objects_Per_Mole*((1)/(1*10**(24))) #ions/nm^3
debye_Length = 1/(np.sqrt(debye_Sum_Scaling*(buffer_Sum_Converted + salt_Sum_Converted))) #nm
kappa = 1/debye_Length #1/nm
protein_Chains = x_Clones*y_Clones*z_Clones


print("Debye Length = " + str(debye_Length))
print("kappa = " + str(kappa))
print("LJ Epsilon = " + str(LJ_Potential_Epsilon))
print("LJ Epsilon Converted = " + str(LJ_Potential_Epsilon_Converted))
print("Yukawa Epsilon Scaling = " + str(yukawa_Epsilon_Scaling))
print("Spring Constant = " + str(harmonic_Spring_Constant))
print("Boltzmann Constant Converted = " + str(boltzmann_Constant_Converted))

aa_Stats = pd.read_csv(working_Input_Directory + "Amino_Acid_Coarse_Grained_Stats.csv")
aa_Long_Names = aa_Stats["Amino Acid Long"]
aa_Short_Names = aa_Stats["Amino Acid Short"]
aa_Mass = aa_Stats["Mass"]
aa_Charge = aa_Stats["Charge"]
aa_Sigma = aa_Stats["Sigma"]
aa_Lambda = aa_Stats["Lambda"]

protein_Chain = pd.read_csv(working_Input_Directory + "TDP_43_LCD_" + str(phos.replace("_","")) + "_Phos_Chain.csv")
chain_Length = len(protein_Chain)
number_Of_Bonds = chain_Length - 1
aa_Chain_Masses = np.zeros(chain_Length)
aa_Chain_Charges = np.zeros(chain_Length)
aa_Chain_Sigmas = np.zeros(chain_Length)
aa_Chain_Lambdas = np.zeros(chain_Length)
aa_Chain_Indices = np.zeros(chain_Length)

for i in range(chain_Length):
  index_Match = aa_Stats[aa_Stats["Amino Acid Short"] == protein_Chain["Amino Acid Short"][i]].index[0]
  aa_Chain_Masses[i] = aa_Mass[index_Match]
  aa_Chain_Charges[i] = aa_Charge[index_Match]
  aa_Chain_Sigmas[i] = aa_Sigma[index_Match]
  aa_Chain_Lambdas[i] = aa_Lambda[index_Match]
  aa_Chain_Indices[i] = index_Match

### Run the following code to initialize a single protein in the simulation box.

The following code initializes a single protein in the simulation box according to the chain structure. Furthermore, the code initializes bonds between all pairs of particles next to one another in the chain.

In [ ]:
snapshot = gsd.hoomd.Snapshot()
snapshot.particles.N = chain_Length
snapshot.particles.types = aa_Short_Names.values.tolist()
snapshot.particles.typeid = aa_Chain_Indices
snapshot.particles.mass = aa_Chain_Masses
snapshot.particles.charge = aa_Chain_Charges

positions=[]
for i in range(chain_Length):
    positions.append((0,0,(i-int(chain_Length/2))*harmonic_Bond_Length))
positions=np.array(positions)
snapshot.particles.position= positions

snapshot.bonds.N = number_Of_Bonds
snapshot.bonds.types = ["Amino_Acid_Bond"]
snapshot.bonds.typeid = [0]*snapshot.bonds.N
bond_Pairs=np.zeros((number_Of_Bonds,2))
for i in range(number_Of_Bonds):
    bond_Pairs[i,:]=np.array([i,i+1])
snapshot.bonds.group = bond_Pairs

box_Dimensions = np.ceil(chain_Length*harmonic_Bond_Length) + 10
snapshot.configuration.box = [box_Dimensions, box_Dimensions, box_Dimensions, 0, 0, 0]

with gsd.hoomd.open(name= working_Output_Directory + "5x_Protein_" + str(protein_Concentration) + "_uM_Protein_" + str(int(salt_Concentration*10**(3))) + "_mM_Salt_" + str(phos) + "_phos_" + str(LJ_Potential_Epsilon) + "_Epsilon_single_Protein.gsd", mode='wb') as f:
  f.append(snapshot)

### Run the following code to clone the single protein into many proteins.

The following code clones the initial protein into many proteins.

In [ ]:
c = hoomd.context.initialize("")
system_State = hoomd.init.read_gsd(working_Output_Directory + "5x_Protein_" + str(protein_Concentration) + "_uM_Protein_" + str(int(salt_Concentration*10**(3))) + "_mM_Salt_" + str(phos) + "_phos_" + str(LJ_Potential_Epsilon) + "_Epsilon_single_Protein.gsd")
system_State.replicate(nx = x_Clones, ny = y_Clones, nz = z_Clones)
snapshot = system_State.take_snapshot(particles = True, bonds = True)
hoomd.dump.gsd(filename = working_Output_Directory + "5x_Protein_" + str(protein_Concentration) + "_uM_Protein_" + str(int(salt_Concentration*10**(3))) + "_mM_Salt_" + str(phos) + "_phos_" + str(LJ_Potential_Epsilon) + "_Epsilon_cloned_Proteins.gsd", group = hoomd.group.all(),period = None)

### Run the following code to compress the cloned protein system into a smaller initial condition to induce a "semi phase-separated" state.

The following code will run a mini simulation to squish the cloned proteins into a small initial condition box, which will induce a "semi phase-separated" state of the proteins when we run the final simulation. This step reduces the amount of time required to observe phase separation of a system.

In [ ]:
c = hoomd.context.initialize("")
sim = hoomd.init.read_gsd(working_Output_Directory + "5x_Protein_" + str(protein_Concentration) + "_uM_Protein_" + str(int(salt_Concentration*10**(3))) + "_mM_Salt_" + str(phos) + "_phos_" + str(LJ_Potential_Epsilon) + "_Epsilon_cloned_Proteins.gsd")
snapshot = sim.take_snapshot(particles = True, bonds = True)

cell = hoomd.md.nlist.cell()
cell.reset_exclusions(exclusions=['1-2', 'body'])

yukawa = hoomd.md.pair.yukawa(r_cut=0.0, nlist=cell)
for aa_One in aa_Short_Names:
  charge_One = aa_Stats[aa_Stats["Amino Acid Short"] == aa_One]["Charge"].iloc[0]
  for aa_Two in aa_Short_Names:
    charge_Two = aa_Stats[aa_Stats["Amino Acid Short"] == aa_Two]["Charge"].iloc[0]
    yukawa.pair_coeff.set(aa_One,aa_Two,epsilon=charge_One*charge_Two*yukawa_Epsilon_Scaling,kappa = kappa, 
                          r_cut = yukawa_Cutoff_Distance)

ashbaugh = azplugins.pair.ashbaugh(r_cut=0, nlist=cell)
for aa_One in aa_Short_Names:
  sigma_One = aa_Stats[aa_Stats["Amino Acid Short"] == aa_One]["Sigma"].iloc[0]/10 #nm
  lambda_One = aa_Stats[aa_Stats["Amino Acid Short"] == aa_One]["Lambda"].iloc[0]
  for aa_Two in aa_Short_Names:
    sigma_Two = aa_Stats[aa_Stats["Amino Acid Short"] == aa_Two]["Sigma"].iloc[0]/10 #nm
    lambda_Two = aa_Stats[aa_Stats["Amino Acid Short"] == aa_Two]["Lambda"].iloc[0]
    ashbaugh.pair_coeff.set(aa_One,aa_Two,lam=(lambda_One+lambda_Two)/2.,
                      epsilon=LJ_Potential_Epsilon_Converted, sigma=(sigma_One+sigma_Two)/2.,
                            r_cut=ashbaugh_Cutoff_Distance) 

harmonic=hoomd.md.bond.harmonic()
harmonic.bond_coeff.set('Amino_Acid_Bond',k=harmonic_Spring_Constant,r0=harmonic_Bond_Length)


hoomd.md.integrate.mode_standard(dt=time_Step) # Time units in ps
kTinput=temperature * boltzmann_Constant_Converted
integrator = hoomd.md.integrate.langevin(group=hoomd.group.all(), kT=kTinput, seed=0)

hoomd.update.box_resize(Lx=hoomd.variant.linear_interp([(0,sim.box.Lx),(resizing_Steps,box_Size)]),
                        Ly=hoomd.variant.linear_interp([(0,sim.box.Ly),(resizing_Steps,box_Size)]),
                        Lz=hoomd.variant.linear_interp([(0,sim.box.Lz),(resizing_Steps,squish_Box_Z)]),scale_particles=True)

for aa_One in aa_Short_Names:
  integrator.set_gamma(aa_One,aa_Stats[aa_Stats["Amino Acid Short"] == aa_One]["Mass"].iloc[0]/1000)

hoomd.run(tsteps=resizing_Steps)

resized_Snapshot = sim.take_snapshot(particles = True, bonds = True)

hoomd.dump.gsd(filename = working_Output_Directory + "5x_Protein_" + str(protein_Concentration) + "_uM_Protein_" + str(int(salt_Concentration*10**(3))) + "_mM_Salt_" + str(phos) + "_phos_" + str(LJ_Potential_Epsilon) + "_Epsilon_compressed_System.gsd", group = hoomd.group.all(),period = None)

### Run the following chunk of code to extend the squished simulation box to the final slab formation.

The following code extends the simulation box to the final slab configuration for final simulations. Note, the fix_Periodic_Harmonic_Errors() function and its use. Particles can "see" each other through the edges of the simulation box, and when we extend the box along the z-axis, particles that were once "close" to one another, but on different sides of the z-boundary are now quite far from one another. This can result in code instability, because at early timesteps in the final simulation, the harmonic bond potential is VERY large, resulting in particles leaving the simulation box. The fix_Periodic_Harmonic_Errors() function fixes this problem.

In [ ]:
hoomd.context.initialize("")
sim = hoomd.init.read_gsd(working_Output_Directory + "5x_Protein_" + str(protein_Concentration) + "_uM_Protein_" + str(int(salt_Concentration*10**(3))) + "_mM_Salt_" + str(phos) + "_phos_" + str(LJ_Potential_Epsilon) + "_Epsilon_compressed_System.gsd")
snapshot = sim.take_snapshot(particles = True, bonds = True)
resized_Snapshot = sim.take_snapshot(particles = True, bonds = True)

def fix_Periodic_Harmonic_Errors(s):
    zmin,zmax,dz = -s.box.Lz/2., s.box.Lz/2., s.box.Lz
    pos1 =  s.particles.position
    pos = pos1.copy()
    skip=0
    ncomp=1
    for k in range(ncomp):
        nchain = int(s.particles.N/chain_Length)
        nres = chain_Length
        for i in range(nchain):
            mol_coord = pos[i*nres+skip:(i+1)*nres+skip,2]
            for j in range(1,nres):
                dist2 = (mol_coord[j] - mol_coord[j-1])**2
                if dist2 > 8:
                    excess = np.sign(mol_coord[j] - mol_coord[j-1])*dz
                    mol_coord[j] = mol_coord[j] - excess 
                com = np.mean(mol_coord)
                if com < zmin:
                    mol_coord += dz
                elif com > zmax:
                    mol_coord -= dz
            pos[i*nres+skip:(i+1)*nres+skip,2] = mol_coord
        skip += nchain*nres
    return pos

snapshot = gsd.hoomd.Snapshot()
snapshot.particles.N = resized_Snapshot.particles.N
snapshot.particles.types = resized_Snapshot.particles.types 
snapshot.particles.typeid = resized_Snapshot.particles.typeid 
snapshot.particles.mass = resized_Snapshot.particles.mass
snapshot.particles.charge = resized_Snapshot.particles.charge
snapshot.particles.position = fix_Periodic_Harmonic_Errors(resized_Snapshot)
snapshot.bonds.N = resized_Snapshot.bonds.N
snapshot.bonds.types = resized_Snapshot.bonds.types
snapshot.bonds.typeid = resized_Snapshot.bonds.typeid
snapshot.bonds.group = resized_Snapshot.bonds.group
snapshot.configuration.box = [box_Size,box_Size,slab_Z_Length,0,0,0] 
snapshot.configuration.step = 0
outfile = gsd.hoomd.open(working_Output_Directory + "5x_Protein_" + str(protein_Concentration) + "_uM_Protein_" + str(int(salt_Concentration*10**(3))) + "_mM_Salt_" + str(phos) + "_phos_" + str(LJ_Potential_Epsilon) + "_Epsilon_compressed_Extended_Box.gsd",'wb')
outfile.append(snapshot)
outfile.close()

### Run the following chunk of code to run the final simulation of the protein system.

The following chunk of code runs the final simulation for however many timesteps the user wants as specified in the first chunk of code.

In [ ]:
c = hoomd.context.initialize("");
sim = hoomd.init.read_gsd(working_Output_Directory + "5x_Protein_" + str(protein_Concentration) + "_uM_Protein_" + str(int(salt_Concentration*10**(3))) + "_mM_Salt_" + str(phos) + "_phos_" + str(LJ_Potential_Epsilon) + "_Epsilon_compressed_Extended_Box.gsd")
snapshot = sim.take_snapshot(particles = True, bonds = True)

cell = hoomd.md.nlist.cell()
cell.reset_exclusions(exclusions=['bond', 'body'])

yukawa = hoomd.md.pair.yukawa(r_cut=0.0, nlist=cell)

for aa_One in aa_Short_Names:
  charge_One = aa_Stats[aa_Stats["Amino Acid Short"] == aa_One]["Charge"].iloc[0]
  for aa_Two in aa_Short_Names:
    charge_Two = aa_Stats[aa_Stats["Amino Acid Short"] == aa_Two]["Charge"].iloc[0]
    yukawa.pair_coeff.set(aa_One,aa_Two,epsilon=charge_One*charge_Two*yukawa_Epsilon_Scaling,kappa = kappa,
                          r_cut = yukawa_Cutoff_Distance)

ashbaugh = azplugins.pair.ashbaugh(r_cut=0, nlist=cell)
for aa_One in aa_Short_Names:
  sigma_One = aa_Stats[aa_Stats["Amino Acid Short"] == aa_One]["Sigma"].iloc[0]/10 #nm
  lambda_One = aa_Stats[aa_Stats["Amino Acid Short"] == aa_One]["Lambda"].iloc[0]
  for aa_Two in aa_Short_Names:
    sigma_Two = aa_Stats[aa_Stats["Amino Acid Short"] == aa_Two]["Sigma"].iloc[0]/10 #nm
    lambda_Two = aa_Stats[aa_Stats["Amino Acid Short"] == aa_Two]["Lambda"].iloc[0]
    ashbaugh.pair_coeff.set(aa_One,aa_Two,lam=(lambda_One+lambda_Two)/2.,
                      epsilon=LJ_Potential_Epsilon_Converted, sigma=(sigma_One+sigma_Two)/2.,r_cut=ashbaugh_Cutoff_Distance)

harmonic=hoomd.md.bond.harmonic()
harmonic.bond_coeff.set('Amino_Acid_Bond',k=harmonic_Spring_Constant,r0=harmonic_Bond_Length)


hoomd.md.integrate.mode_standard(dt=time_Step) # Time units in ps
kTinput=temperature * boltzmann_Constant_Converted
integrator = hoomd.md.integrate.langevin(group=hoomd.group.all(), kT=kTinput, seed=0)


for aa_One in aa_Short_Names:
  integrator.set_gamma(aa_One,aa_Stats[aa_Stats["Amino Acid Short"] == aa_One]["Mass"].iloc[0]/1000)

hoomd.dump.gsd(working_Output_Directory + "5x_Protein_" + str(protein_Concentration) + "_uM_Protein_" + str(int(salt_Concentration*10**(3))) + "_mM_Salt_" + str(phos) + "_phos_" + str(LJ_Potential_Epsilon) + "_Epsilon_Trajectory.gsd", period=10**5, group=hoomd.group.all(), truncate=False, overwrite = True)

hoomd.run(production_Steps)